#Background

This is a experimentation setup to use LLaMA3 LLM, Langchain and Neo4j, to create a GraphRAG Setup and execute queries to draw insights.

##Steps at a Glance

1. Install Required Dependencies
2. Create a Neo4j Account and Start a Aura DB Instance
3. Setup Credentials and LLM
4. Setup Conversation Chat History
5. Download Cricket Data
6. Read the Cricket Data using Langchain
7. Create Vector Indexes for the Data 
8. Execute Queries
9. Experimentation
10. Reference Documentation

##Step 1.

Install the required Neo4j and Langchanin dependencies. 

[Prerequisite] Please Install Ollama to execute the LLaMA LLM, and Setup Jupyter Notebooks locally by following Step 1 & 2 in the following Notebook - llama-langchain-experimentaion.ipynb

In [1]:
!pip install --upgrade --quiet neo4j yfiles_jupyter_graphs_for_neo4j

In [2]:
!pip install --upgrade --quiet  json-repair networkx langchain-core langchain-experimental langchain-community

##2. Create a Neo4j Account and Start an Aura DB Instance

Go To: https://neo4j.com/product/auradb/ and Click on `Get Started for Free`
Create an Account and Login. Alternatively use your Gmail creentials to Signup & Login.
Create a Free AuraDB Instance and wait for it to start up. 

Please Note: Ensure to download/copy the credentials for the AuraDB Instance and store it safely.

##3. Setup Credentials and LLM

In [3]:
from langchain_community.chat_models import ChatOllama

url = "YOUR NEO4J AURADB CONNECTION URL"
username = "neo4j"
password = "YOUR NEO4J AURADB PASSWORD"

llm = ChatOllama(model="llama3.1", temperature=0) 

##4. Setup text to be used to create the Knowledge Graph

If the `useStaticText` flag is turned off, it will pull information from a website and split the data into chunks for processing.

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

useStaticText = True
docs = []

if useStaticText:
    text = """
    Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
    She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
    Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
    She was, in 1906, the first woman to become a professor at the University of Paris. 
    """
    docs = [Document(page_content=text)]

else:
    loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

USER_AGENT environment variable not set, consider setting it to identify your requests.


##5. Create the Knowledge Graph

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(docs)

# Congigurations can be added as below to control the Kind of Nodes and Relationships in the KNowledge Graph.
#
# llm_transformer_filtered = LLMGraphTransformer(
#     llm=llm,
#     allowed_nodes=["Person", "Country", "Organization"],
#     allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
# )
# graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
#     d
# )

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Nobel Prize', type='Award', properties={}), Node(id='five Nobel Prizes', type='Award', properties={}), Node(id='radioactivity', type='Field of Study', properties={}), Node(id='Marie Curie', type='Person', properties={}), Node(id='two scientific fields', type='Field of Study', properties={}), Node(id='physicist and chemist', type='Occupation', properties={}), Node(id='University of Paris', type='Institution', properties={}), Node(id="Marie Curie's first Nobel Prize", type='Event', properties={}), Node(id='Pierre Curie', type='Person', properties={}), Node(id='1867', type='Year', properties={})]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='1867', type='Year', properties={}), type='BORN_IN', properties={}), Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='physicist and chemist', type='Occupation', properties={}), type='WORKS_AS', properties={}), Relationship(source=Node(id=

##6. Connect to Neo4j AuraDB and add the Graph

Please change the flag `addGraphDocuments` to `True` when creating the Knowledge Graph for the first time. It is set to `False` to prevent subsequent run for crearing duplicate nodes in the Knowledge Graph.

In [7]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [8]:
addGraphDocuments = True

if addGraphDocuments:
    graph.add_graph_documents(
        graph_documents,
        baseEntityLabel=True, 
        include_source=True
    )

##7. Visualize the Graph

Please Note: If you get an error with the YFiles Widget, please ensure you close the reopen the Jupyter Notebook to ensure the dependencies are loaded correctly.

In [10]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri = url, auth = (username, password))
g = Neo4jGraphWidget(driver)
g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t")

GraphWidget(layout=Layout(height='610px', width='100%'))

##8. Execute a Query

In [20]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=llm, 
    verbose=True,
    return_intermediate_steps=True,
    allow_dangerous_requests=True
)

response = chain.invoke({"query": "what do the people work as?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:WORKS_AS]->(o) RETURN o;
Full Context:
[{'o': {'id': 'physicist and chemist'}}]

> Finished chain.


{'query': 'what do the people work as?',
 'result': 'People work as a physicist and chemist.',
 'intermediate_steps': [{'query': 'MATCH (p:Person)-[:WORKS_AS]->(o) RETURN o;'},
  {'context': [{'o': {'id': 'physicist and chemist'}}]}]}